Introducción rápida a Keras para expertos --- 0:00 min
===

* Última modificación: Marzo 1, 2022 | YouTube

Importación de librerías
---

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf

tf.__version__

'2.8.0'

Descarga de datos
---

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#
# Escalamiento del dataset
#
x_train, x_test = x_train / 255.0, x_test / 255.0

#
# Adición de una dimensión al tensor
#
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

Uso de tf.data.Dataset
---

In [3]:
train_ds = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Especificación del modelo
--

In [4]:
class MyModel(tf.keras.Model):
    #
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation="relu")
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation="relu")
        self.d2 = tf.keras.layers.Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


#
# Creación de una instancia del modelo
#
model = MyModel()

In [5]:
#
# Función de pérdida
#
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#
# Optimizador
#
optimizer = tf.keras.optimizers.Adam()

In [6]:
#
# Métricas
#
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [7]:
#
# Uso de gradient tape
#
@tf.function
def train_step(images, labels):

    with tf.GradientTape() as tape:
        #
        # trainig=true cuando el compotamiento de las capas es diferente
        # durante la optimización y el pronóstico
        #
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
    #
    # trainig=true cuando el compotamiento de las capas es diferente
    # durante la optimización y el pronóstico
    #
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
    #
    # Reinicio de las métricas durante cada iteración
    #
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f"Epoch {epoch + 1}, "
        f"Loss: {train_loss.result()}, "
        f"Accuracy: {train_accuracy.result() * 100}, "
        f"Test Loss: {test_loss.result()}, "
        f"Test Accuracy: {test_accuracy.result() * 100}"
    )

Epoch 1, Loss: 0.14401187002658844, Accuracy: 95.67499542236328, Test Loss: 0.05975964665412903, Test Accuracy: 98.18000030517578
Epoch 2, Loss: 0.04479236900806427, Accuracy: 98.61833190917969, Test Loss: 0.054280418902635574, Test Accuracy: 98.16999816894531
Epoch 3, Loss: 0.02204476110637188, Accuracy: 99.31999969482422, Test Loss: 0.05362352728843689, Test Accuracy: 98.36000061035156
Epoch 4, Loss: 0.01368221826851368, Accuracy: 99.53500366210938, Test Loss: 0.05262510105967522, Test Accuracy: 98.44999694824219
Epoch 5, Loss: 0.009993226267397404, Accuracy: 99.66333770751953, Test Loss: 0.06346754729747772, Test Accuracy: 98.3699951171875
